In [ ]:
from IPython.display import display, HTML

html = """
<div style="background:linear-gradient(135deg,#1a1a2e,#16213e,#0f3460);border-radius:16px;padding:40px;text-align:center;font-family:'Segoe UI',Arial,sans-serif;box-shadow:0 8px 32px rgba(0,0,0,0.4);">
  <div style="font-size:2.5em;">⚙️</div>
  <h1 style="color:#e0e0e0;font-size:2em;margin:8px 0;">Preprocesamiento de Texto</h1>
  <p style="color:#a0aec0;letter-spacing:2px;text-transform:uppercase;font-size:0.95em;">Limpieza · Lematizacion · TF-IDF · Vectorizacion</p>
  <div style="background:rgba(255,255,255,0.06);border:1px solid rgba(255,255,255,0.1);border-radius:10px;padding:16px 24px;max-width:650px;margin:24px auto 0;text-align:left;color:#cbd5e0;font-size:0.93em;line-height:1.7;">
    Limpiamos los textos legales, aplicamos lematizacion y los transformamos en
    vectores numericos con TF-IDF para que los modelos puedan procesarlos.
    Guardamos los artefactos para ser reutilizados en el notebook de modelado.
  </div>
</div>
"""
display(HTML(html))


## 1. Carga de Datos

In [ ]:
import pandas as pd
df = pd.read_csv("../data/raw/legal_text_classification.csv")
df = df.dropna(subset=["case_text"]).reset_index(drop=True)
print(f"Documentos cargados: {len(df):,}")
df.head(3)


## 2. Limpieza con Lematizacion

Convertimos a minusculas, eliminamos puntuacion y numeros, quitamos stopwords y aplicamos lematizacion para reducir palabras a su forma base ("exercised" → "exercise").

In [ ]:
import re
import nltk
from nltk.corpus import stopwords
from nltk.stem import WordNetLemmatizer

nltk.download("stopwords")
nltk.download("wordnet")
nltk.download("omw-1.4")

stop_words = set(stopwords.words("english"))
lemmatizer = WordNetLemmatizer()

def clean_text(text):
    text = str(text).lower()
    text = re.sub(r"[^a-zA-Z\s]", "", text)
    tokens = text.split()
    tokens = [lemmatizer.lemmatize(t) for t in tokens if t not in stop_words and len(t) > 2]
    return " ".join(tokens)

df["clean_text"] = df["case_text"].astype(str).apply(clean_text)
df[["case_text", "clean_text"]].head(3)


## 3. Palabras mas Frecuentes

In [ ]:
from collections import Counter
import matplotlib.pyplot as plt

all_words = " ".join(df["clean_text"]).split()
freq = Counter(all_words)
common = freq.most_common(20)
words, counts = zip(*common)

plt.figure(figsize=(11, 5))
plt.bar(words, counts, color="#4299e1", alpha=0.85)
plt.xticks(rotation=65)
plt.title("Top 20 palabras mas frecuentes (texto limpio)")
plt.tight_layout()
plt.savefig("../reports/prep_palabras_frecuentes.png", dpi=150, bbox_inches="tight")
plt.show()


## 4. Vectorizacion TF-IDF

Usamos bigramas para capturar frases como "referred to" o "applied for". `sublinear_tf=True` suaviza el peso de terminos muy frecuentes.

In [ ]:
from sklearn.feature_extraction.text import TfidfVectorizer
import joblib, os

vectorizer = TfidfVectorizer(
    max_features=10000,
    ngram_range=(1, 2),
    min_df=3,
    max_df=0.85,
    sublinear_tf=True
)

X = vectorizer.fit_transform(df["clean_text"])
y = df["case_outcome"]

print(f"Matriz TF-IDF: {X.shape}")
print(f"Documentos: {X.shape[0]:,}  |  Features: {X.shape[1]:,}")
print(f"\nDistribucion de clases:\n{y.value_counts()}")


## 5. Guardado de Artefactos

Guardamos el vectorizador y el dataframe limpio para que el notebook de modelado los pueda reutilizar sin reprocesar.

In [ ]:
import joblib, os
import pickle

os.makedirs("../models", exist_ok=True)

# Guardar vectorizador
joblib.dump(vectorizer, "../models/tfidf_vectorizer.pkl")

# Guardar matriz X, vector y y dataframe limpio
joblib.dump(X, "../models/X_tfidf.pkl")
joblib.dump(y, "../models/y_labels.pkl")
df[["case_text", "clean_text", "case_outcome"]].to_csv("../data/raw/df_limpio.csv", index=False)

print("Artefactos guardados en ../models/:")
for f in os.listdir("../models"):
    size = os.path.getsize(f"../models/{f}") / 1024
    print(f"  {f}  ({size:.1f} KB)")
